In [1]:
USE_EASYOCR = 1

if 1 == 0:
    %pip install easyocr  
    %pip install pytesseract 
    %pip install opencv-python
    %pip install tensorflow

In [2]:
import cv2
from PIL import Image
import tensorflow as tf
import numpy as np
if USE_EASYOCR == 1:
    import easyocr
else:
    import pytesseract

model_path = 'model.tflite'
ocr_config = r'-l ben --oem 3 --psm 11'

ModuleNotFoundError: No module named 'easyocr'

In [ ]:
# Load the labels into a list
classes = []
with open('labels.txt', 'r') as f:
  for line in f:
    if 'str' in line or line == '': break
    lbl = line.replace('\n', '')
    classes.append(lbl)

In [ ]:
# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

In [ ]:
def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  return resized_img, original_image

In [ ]:
def set_input_tensor(interpreter, image):
  """Set the input tensor."""
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image

In [ ]:
def get_output_tensor(interpreter, index):
  """Retur the output tensor at the given index."""
  output_details = interpreter.get_output_details()[index]
  tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
  return tensor

In [ ]:
def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""
  # Feed the input image to the model
  set_input_tensor(interpreter, image)
  interpreter.invoke()

  # Get all outputs from the model
  scores = get_output_tensor(interpreter, 0)
  boxes = get_output_tensor(interpreter, 1)
  count = int(get_output_tensor(interpreter, 2))
  classes = get_output_tensor(interpreter, 3)

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results

In [ ]:
def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path, 
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  print('DETECTED NUMBER PLATE:')
  print('----------------------') 
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute 
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * (original_image_np.shape[1] - 5))
    xmax = int(xmax * (original_image_np.shape[1] + 10))
    ymin = int(ymin * (original_image_np.shape[0] - 5))
    ymax = int(ymax * (original_image_np.shape[0] + 10))

    # Find the class index of the current object
    class_id = int(obj['class_id'])
    
    # crop image and apply OCR process
    w = xmax - xmin
    h = ymax - ymin
    cropped = original_image_np[ymin:ymin+h, xmin:xmin+w]
    cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    (_, blackWhite) = cv2.threshold(cropped, 100, 255, cv2.THRESH_BINARY)
    detected_img = Image.fromarray(blackWhite)
    detected_img.save('detected_plate.jpg')
    
    if USE_EASYOCR == 1:
      reader = easyocr.Reader(['bn'])
      result = reader.readtext('detected_plate.jpg', paragraph="False")
      print(result[0][1])
    else:
      number = pytesseract.image_to_string(blackWhite, config=ocr_config)
      print(number)

    # Draw the bounding box and label on the image
    color = [255, 255, 0]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 1)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_PLAIN, 1.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
INPUT_IMAGE_URL = "sample/car3.jpg"
DETECTION_THRESHOLD = 0.5 

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    INPUT_IMAGE_URL, 
    interpreter, 
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
Image.fromarray(detection_result_image)